In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [84]:
# poem generation
# Deep learning: a combination of VAE/GAN
# condition the generator of the GAN for rubustness
# latent space!!
# how do you evaluate these?
# discriminator

# test using a model on text and poems to predict whether a given text is a poem or not.
# maybe no input!
# my goal will be to generate 10,000 poems


# future project would be to write books using ai. minimum of 10-50 pages based on the most popular topic on xyz site?
# based on stars guess where the picture was taken in space?

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging
import re
import contractions
import nltk

In [5]:
df = pd.read_pickle('mostly_english_poems.pickle')
print(len(df))

26817


In [6]:
poems = df.content

In [41]:
def remove_punc(dirty_line) -> str:
    line = dirty_line.encode('ascii', 'ignore')
    line = line.decode()
    line = line.replace("'", '')
    line = re.sub('[&,.”‘-“;@:"\'„—#“‛-‟…?!‚’_)–(*)$]+', ' ', line)
    line = line.replace('-', ' ')
    line = re.sub('\s\s+' , ' ', line)
    return line

In [31]:
def clean_poems(list_of_words) -> list[str]:
    cleaned_poems = []
    for string in list_of_words:

        lines = string.split('\n')
        good_lines = ''

        for line in lines:
            line = line.replace('\'', "'")
            line = line.lower() + ' NEWLINE '
            # replaces contractions with their expanded version: can't -> cannot
            line = contractions.fix(line)
            # removes punctuation from the text and makes words lowercase
            line = remove_punc(line)
            
            good_lines += line 

        # removes the ' NEWLINE ' from the end of the poem
        cleaned_poems.append(good_lines[:len(good_lines)-9])
        
    return cleaned_poems

In [ ]:
# use gingerit to fix grammar after? idk

In [42]:
cl_poems = clean_poems(poems[:100])

In [43]:
cl_poems

['do not go gentle into that good night NEWLINE old age should burn and rave at close of day NEWLINE rage rage against the dying of the light NEWLINE though wise men at their end know dark is right NEWLINE because their words had forked no lightning they NEWLINE do not go gentle into that good night NEWLINE good men the last wave by crying how bright NEWLINE their frail deeds might have danced in a green bay NEWLINE rage rage against the dying of the light NEWLINE wild men who caught and sang the sun in flight NEWLINE and learn too late they grieved it on its way NEWLINE do not go gentle into that good night NEWLINE grave men near death who see with blinding sight NEWLINE blind eyes could blaze like meteors and be gay NEWLINE rage rage against the dying of the light NEWLINE and you my father there on the sad height NEWLINE curse bless me now with your fierce tears i pray NEWLINE do not go gentle into that good night NEWLINE rage rage against the dying of the light',
 'how do i love the

In [92]:
for i in range(len(poems)):
    if 'translated by ' in poems[i]:
        print(poems[i])

translated by NEWLINE roderick gill NEWLINE o faint remembrances of vanished days NEWLINE that stole away on such a velvet wing NEWLINE over meads and groves over plains and mountain ways  NEWLINE what grief and sorrow to my heart you bring  NEWLINE come back without the shadow of your care  NEWLINE come back in silence and without a moan  NEWLINE as the birds cross the unregarding air NEWLINE till none may tell the whence or whither flown  NEWLINE come back amid the pallor of the moon NEWLINE that silvers all the azure rifts at sea  NEWLINE or in the deadly mist that in a swoon NEWLINE engulfs afar the green palm’s royal tree  NEWLINE bring back the murmur of the doves that made NEWLINE their little nests so neighborly to mine  NEWLINE the vibrant airs––the fragrances that played NEWLINE around the peaks that saw my cradle shine  NEWLINE sing in my ear the melodies of old  NEWLINE so sweet and joyous to my inmost heart  NEWLINE o faint remembrances two breasts should hold  NEWLINE two

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oliver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
Poems = df.content.map(lambda x: nltk.tokenize.word_tokenize(x))

In [18]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)